In [1]:
from torchvision.models import convnext_tiny as model 
from torchvision.models import ConvNeXt_Tiny_Weights as model_weights
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

In [2]:
def imshow(img):
    plt.figure(figsize=(5, 5))
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    plt.show()

In [3]:
# Cargar el modelo preentrenado de DenseNet121
weights = model_weights.IMAGENET1K_V1
model = model(weights=weights)
model.eval()  # Poner el modelo en modo de evaluación

# Definir las transformaciones para la imagen
preprocess = transforms.Compose([
    transforms.Resize(256),  # Redimensionar la imagen a 256x256 píxeles
    transforms.CenterCrop(224),  # Recortar la imagen al centro a 224x224 píxeles
    transforms.ToTensor(),  # Convertir la imagen a un tensor de PyTorch
    transforms.Normalize(mean=weights.transforms().mean, std=weights.transforms().std),  # Normalizar la imagen
])

# Diccionario para mapear nombres de carpetas a labels personalizados
custom_labels = {
    'dolphin': 148,
    'eagle': 22,
    'falcon': 21,
    'labrador': 208,
    'lion': 291,
    'persian': 283,
    'shark': 2,
    'tabby': 281,
    'tiger': 292,
    'wolf': 269,
}

# Función para transformar los targets/labels
def custom_label_transform(target):
    # Obtén el label original (nombre de la carpeta) del dataset
    folder_name = dataset.classes[target]
    # Retorna el label personalizado usando el diccionario
    return custom_labels.get(folder_name, folder_name)  # Retorna el nombre de la carpeta si no se encuentra en el diccionario

# Cargando el dataset con labels personalizados
dataset = datasets.ImageFolder('../imagenet_data/', transform=preprocess, target_transform=custom_label_transform)


class BalancedDataset(Dataset):
    def __init__(self, dataset, num_samples_per_class=2000):
        self.dataset = dataset
        self.num_samples_per_class = num_samples_per_class
        
        # Organizar los datos por clase
        self.indices_per_class = {}
        for idx, (_, label) in enumerate(self.dataset):
            if label not in self.indices_per_class:
                self.indices_per_class[label] = []
            self.indices_per_class[label].append(idx)
        
        # Seleccionar aleatoriamente num_samples_per_class índices para cada clase
        self.balanced_indices = []
        for label, indices in self.indices_per_class.items():
            if len(indices) >= num_samples_per_class:
                self.balanced_indices.extend(random.sample(indices, num_samples_per_class))
            else:
                # Si hay menos de num_samples_per_class, reutiliza algunos índices
                self.balanced_indices.extend(indices * (num_samples_per_class // len(indices)) + random.sample(indices, num_samples_per_class % len(indices)))
        
        # Mezclar los índices para asegurar aleatoriedad en el DataLoader
        random.shuffle(self.balanced_indices)
    
    def __getitem__(self, index):
        # Obtener el índice original del conjunto de datos
        original_index = self.balanced_indices[index]
        return self.dataset[original_index]
    
    def __len__(self):
        return len(self.balanced_indices)

# Suponiendo que `original_dataset` es tu conjunto de datos original
balanced_dataset = BalancedDataset(dataset)

# Crear un DataLoader para cargar los datos en lotes
dataloader = torch.utils.data.DataLoader(balanced_dataset, batch_size=64, shuffle=True)


In [4]:
#images, labels = next(iter(dataloader))
#print("Pred:", model(images).max(1).indices)
#print("Real:", labels)
##imshow(torchvision.utils.make_grid(images))

In [5]:
def hook(module, input, output):
    tensors.append(output.clone().detach())
model.classifier[1].register_forward_hook(hook)

In [6]:
y = []
used_images = []
tensors = []

for images, labels in tqdm(dataloader, desc='Processing'):
    used_images.append(images)
    y.append(labels)
    model(images)

Processing:   0%|          | 0/313 [00:00<?, ?it/s]

Processing: 100%|██████████| 313/313 [24:59<00:00,  4.79s/it]


In [7]:
used_images = torch.cat(used_images, dim=0)
y = torch.cat(y, dim=0).tolist()
tensors = torch.cat(tensors, dim=0)

torch.save(used_images, "images.pth")
torch.save(tensors, "tensors.pth")
torch.save(y, "labels.pth")